In [218]:
# Imports
import requests
import pandas as pd

In [219]:
# Global Variables
api_key = "RGAPI-cb7dc070-2672-45bc-ad67-fdfeaa63a8d4"
base_url = "https://americas.api.riotgames.com"
tag_line = "NA1"

### Summoner Functions

In [220]:
# Function to get unique PUUID by summoner name
def get_puuid_by_name(gameName, tag_line):
    try:
        url = f"https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tag_line}"
        response = requests.get(url, headers={"X-Riot-Token": api_key})
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        if "puuid" not in data:
            raise KeyError("PUUID not found in response")
        return data["puuid"]
    except requests.exceptions.RequestException as e:
        print(f"Error making request for {gameName}#{tag_line}: {str(e)}")
        return None
    except (KeyError, ValueError) as e:
        print(f"Error parsing response for {gameName}#{tag_line}: {str(e)}")
        return None

# Function to get summoner data by PUUID
def get_summoner_by_puuid(puuid):
    try:
        if not puuid:
            raise ValueError("PUUID cannot be empty")
        url = f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
        response = requests.get(url, headers={"X-Riot-Token": api_key})
        response.raise_for_status()
        data = response.json()
        if not data:
            raise ValueError("Empty response received")
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error making request for PUUID {puuid}: {str(e)}")
        return None
    except ValueError as e:
        print(f"Error processing summoner data for PUUID {puuid}: {str(e)}")
        return None

# Function to get rank data by PUUID
def get_rank_by_summoner_id(puuid):
    try:
        if not puuid:
            raise ValueError("PUUID cannot be empty")
        url = f"https://na1.api.riotgames.com/lol/league/v4/entries/by-puuid/{puuid}"
        response = requests.get(url, headers={"X-Riot-Token": api_key})
        response.raise_for_status()
        data = response.json()
        if not isinstance(data, list):
            raise ValueError("Expected list response for rank data")
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error making request for PUUID {puuid}: {str(e)}")
        return None
    except ValueError as e:
        print(f"Error processing rank data for PUUID {puuid}: {str(e)}")
        return None

### Match Functions

In [223]:
# Function to get match history by PUUID
def get_matches_by_puuid(puuid):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?count=100"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    matches = response.json()
    return matches

# Function to get match data by match ID
def get_match_data_by_match_id(match_id):
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}"
    response = requests.get(url, headers={"X-Riot-Token": api_key})
    match_data = response.json()
    return match_data

# Function to get the PUUIDs of the participants in a match
def get_match_puuids(match_id):
    try:
        match_data = get_match_data_by_match_id(match_id)
        if "metadata" not in match_data:
            raise KeyError("Match data does not contain metadata")
        if "participants" not in match_data["metadata"]:
            raise KeyError("Match metadata does not contain participants")
        return match_data["metadata"]["participants"]
    except Exception as e:
        print(f"Error getting match PUUIDs for match {match_id}: {str(e)}")
        return []

# Function to check if a match is ranked
def check_ranked(match_id):
    try:
        match_data = get_match_data_by_match_id(match_id)
        if "info" not in match_data:
            print(f"Match {match_id} data does not contain info")
            return False
        if "queueId" not in match_data["info"]:
            print(f"Match {match_id} info does not contain queueId")
            return False
        return match_data["info"]["queueId"] == 420
    except Exception as e:
        print(f"Error checking if match {match_id} is ranked: {str(e)}")
        return False

In [224]:
test_puuid = get_puuid_by_name("ARMAO", tag_line)
matches = get_matches_by_puuid(test_puuid)
match_data = get_match_data_by_match_id(matches[0])

print(get_match_puuids(matches[0]))
for match in matches:
    print(f"Checking match {match}")
    if check_ranked(match):
        print(match)



['kmsFAO_77jjUMeGKQ-ajFOx-EXjF7bYBXZ5MYktRPLy1DFqqq5Y7dO3SgHtNVXzQ4Wgc1J2NVgjLZA', '-7A55h5Ap4Bw8XZCFryUhBjT4q8wiIcxR4U_bnCcErLc1Ry0zwHnlnajqNBH3nR-Wwvz5mHJKv9FRQ', 'KZY9vPlc8JNM4HRcsYEpMfn-0RB48gg7sMP_o7ezc0vnIjLfPwrW3JuqB8eIOdxofJgzlgxLCcaJWw', 'sDeS5zC_lb_nIy9QbnpEihKAQZsdCaLQeZ_RP7vY-mBCOFlPZsV3BMSATDty6dZUrY1OmmFMPhLVIw', 'ntNWuQm-xSDMZqTZKKmciU6DAcGZD7_r1D6nhCTHpEiLyhqCssmO5gAvkHtgWuMK4pqt3Yx-E_F34w', 'XGObXx2C5ckGP_Avqvw29YmV_pyj8uR5VaGvDzRUec8Up0TAGrpGgsCwxEsoGC5kwAePOuDmgroZjw', '5qfT43Vod8IuFlD4X8KWZlANpo9IhVl00LOSxco_526ap3a9qqDN56ozIhWa4ZUGuAUWFUR9qlimWw', 'TawUDMlM0jPs111Y_mcZoB7y-5c0Xw-SdVc-xOgBPRD36Uqd7O0rz73OR4ke46Gh7vQPGz5zUPZXLA', 'NmnZXDZMkHV9vjy54eXr6dxIS12VAF8CTbSej4ZR2d0FGa-EApZTESKGAAEz2bjbooQfZnfmsxlO8g', 'KpB95ghFhG_p6IG2eP138IZC354vpvH-ys_giEnoMlTQn0SN5iWgMY9bqvwYDaIl0RTMrPN3za5ZtA']
Checking match NA1_5244520364
Checking match NA1_5244483429
Checking match NA1_5244302021
Checking match NA1_5244270627
Checking match NA1_5244259397
Checking match NA1_5244238459